## Alexnet-CIFAR10

In [ ]:
!wget https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz

In [ ]:
!tar -zvxf cifar-10-binary.tar.gz

In [2]:
from mindspore import context

context.set_context(mode=context.GRAPH_MODE, device_target='CPU', enable_mem_reuse=False)

In [3]:
from easydict import EasyDict as edict

cfg = edict({
    'num_classes': 10,
    'learning_rate': 0.002,
    'momentum': 0.9,
    'epoch_size': 1, # remember that you are using a Docker with only 1 CPU 
    'batch_size': 32,
    'buffer_size': 1000,
    'image_height': 227,
    'image_width': 227,
    'save_checkpoint_steps': 1562,
    'keep_checkpoint_max': 10,
})

data_path="cifar-10-batches-bin"

In [4]:
import mindspore.dataset.transforms.vision.c_transforms as CV
import mindspore.dataset.transforms.c_transforms as C
from mindspore.common import dtype as mstype


import mindspore.dataset as ds

def create_dataset(data_path, batch_size=32, repeat_size=1, status="train"):
    """
    create dataset for train or test
    """
    cifar_ds = ds.Cifar10Dataset(data_path)
    rescale = 1.0 / 255.0
    shift = 0.0

    resize_op = CV.Resize((cfg.image_height, cfg.image_width))
    rescale_op = CV.Rescale(rescale, shift)
    normalize_op = CV.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    if status == "train":
        random_crop_op = CV.RandomCrop([32, 32], [4, 4, 4, 4])
        random_horizontal_op = CV.RandomHorizontalFlip()
    channel_swap_op = CV.HWC2CHW()
    typecast_op = C.TypeCast(mstype.int32)
    cifar_ds = cifar_ds.map(input_columns="label", operations=typecast_op)
    if status == "train":
        cifar_ds = cifar_ds.map(input_columns="image", operations=random_crop_op)
        cifar_ds = cifar_ds.map(input_columns="image", operations=random_horizontal_op)
    cifar_ds = cifar_ds.map(input_columns="image", operations=resize_op)
    cifar_ds = cifar_ds.map(input_columns="image", operations=rescale_op)
    cifar_ds = cifar_ds.map(input_columns="image", operations=normalize_op)
    cifar_ds = cifar_ds.map(input_columns="image", operations=channel_swap_op)

    cifar_ds = cifar_ds.shuffle(buffer_size=cfg.buffer_size)
    cifar_ds = cifar_ds.batch(batch_size, drop_remainder=True)
    cifar_ds = cifar_ds.repeat(repeat_size)
    return cifar_ds

In [11]:

import mindspore.ops.operations as P
import mindspore.nn as nn
from mindspore.common.initializer import TruncatedNormal

def conv(in_channels, out_channels, kernel_size, stride=1, padding=0, pad_mode="valid"):
    weight = weight_variable()
    return nn.Conv2d(in_channels, out_channels,
                     kernel_size=kernel_size, stride=stride, padding=padding,
                     weight_init=weight, has_bias=False, pad_mode=pad_mode)

def fc_with_initialize(input_channels, out_channels):
    weight = weight_variable()
    bias = weight_variable()
    return nn.Dense(input_channels, out_channels, weight, bias)

def weight_variable():
    return TruncatedNormal(0.02)  # 0.02


class AlexNet(nn.Cell):
    """
    Alexnet
    """
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.batch_size = cfg.batch_size
        self.conv1 = conv(3, 96, 11, stride=4)
        self.conv2 = conv(96, 256, 5, pad_mode="same")
        self.conv3 = conv(256, 384, 3, pad_mode="same")
        self.conv4 = conv(384, 384, 3, pad_mode="same")
        self.conv5 = conv(384, 256, 3, pad_mode="same")
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=3, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = fc_with_initialize(6*6*256, 4096)
        self.fc2 = fc_with_initialize(4096, 4096)
        self.fc3 = fc_with_initialize(4096, num_classes)

    def construct(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.conv4(x)
        x = self.relu(x)
        x = self.conv5(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

In [6]:
import mindspore.nn as nn
from mindspore import context
from mindspore.train import Model
from mindspore.nn.metrics import Accuracy
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor

In [7]:
network = AlexNet(cfg.num_classes)
loss = nn.SoftmaxCrossEntropyWithLogits(is_grad=False, sparse=True, reduction="mean")
opt = nn.Momentum(network.trainable_params(), cfg.learning_rate, cfg.momentum)
model = Model(network, loss, opt, metrics={"Accuracy": Accuracy()})  # test

print("============== Starting Training ==============")
ds_train = create_dataset(data_path, cfg.batch_size, cfg.epoch_size, "train")
config_ck = CheckpointConfig(save_checkpoint_steps=cfg.save_checkpoint_steps,
                                 keep_checkpoint_max=cfg.keep_checkpoint_max)
 
ckpoint_cb = ModelCheckpoint(prefix="checkpoint_alexnet", config=config_ck)
model.train(cfg.epoch_size, ds_train, callbacks=[ckpoint_cb, LossMonitor()],
                dataset_sink_mode=False)

============== Starting Training ==============
epoch: 1 step: 1, loss is 2.2953937
epoch: 1 step: 2, loss is 2.3085976
epoch: 1 step: 3, loss is 2.2658746
epoch: 1 step: 4, loss is 2.300441
epoch: 1 step: 5, loss is 2.3148353
epoch: 1 step: 6, loss is 2.316042
epoch: 1 step: 7, loss is 2.297753
epoch: 1 step: 8, loss is 2.303807
epoch: 1 step: 9, loss is 2.3032458
epoch: 1 step: 10, loss is 2.2855701
epoch: 1 step: 11, loss is 2.23874
epoch: 1 step: 12, loss is 2.253618
epoch: 1 step: 13, loss is 2.333591
epoch: 1 step: 14, loss is 2.3551157
epoch: 1 step: 15, loss is 2.2862926
epoch: 1 step: 16, loss is 2.251695
epoch: 1 step: 17, loss is 2.254908
epoch: 1 step: 18, loss is 2.2822368
epoch: 1 step: 19, loss is 2.1970465
epoch: 1 step: 20, loss is 2.3318756
epoch: 1 step: 21, loss is 2.330506
epoch: 1 step: 22, loss is 2.264245
epoch: 1 step: 23, loss is 2.324888
epoch: 1 step: 24, loss is 2.2817268
epoch: 1 step: 25, loss is 2.2624626
epoch: 1 step: 26, loss is 2.2509553
epoch: 1 ste

In [8]:
!ls

Untitled.ipynb			checkpoint_alexnet-graph.meta  home   opt   srv
bin				cifar-10-batches-bin	       lib    proc  sys
boot				cifar-10-binary.tar.gz	       lib64  root  tmp
checkpoint_alexnet-1_1562.ckpt	dev			       media  run   usr
checkpoint_alexnet-1_1875.ckpt	etc			       mnt    sbin  var


In [10]:
# Check the name of the checkpoint file

from mindspore.train.serialization import load_checkpoint, load_param_into_net


param_dict = load_checkpoint("checkpoint_alexnet-1_1875.ckpt")
load_param_into_net(network, param_dict)

ds_eval = create_dataset(data_path, cfg.batch_size, 1,"test")
acc = model.eval(ds_eval, dataset_sink_mode=False)
print("============== Accuracy:{} ==============".format(acc))

============== Accuracy:{'Accuracy': 0.5426666666666666} ==============


Comment related Accuracy: only 1 epoch is executed (required 90 minutes in my computer)